In [1]:
import numpy as np
import random
from math import *
import time

import copy

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR, MultiStepLR

In [2]:
torch.set_default_tensor_type('torch.DoubleTensor')

In [3]:
class Net(torch.nn.Module):
    def __init__(self,input_width,layer_width):
        super(Net,self).__init__()
        self.layer_out = torch.nn.Linear(layer_width, 1)
    def forward(self,x):
        output = self.layer_out(x)
        return output

In [4]:
dimension = 1

In [5]:
input_width,layer_width = dimension, 1

In [6]:
net = Net(input_width,layer_width)

In [7]:
# defination of exact solution
def u_ex(x):     
    u_temp = 2*x**3 - x**2 - x
    return u_temp.reshape([x.size()[0], 1])

In [8]:
# defination of f(x)
def f(x):
    f_temp = -(12*x - 2)
    return f_temp.reshape([x.size()[0],1])

In [9]:
# generate points by random
def generate_sample(data_size):
    sample_temp = torch.rand(data_size, dimension)
    return sample_temp

In [10]:
def model(x):
    x_temp = x
    D_x_0 = torch.prod(x_temp, axis = 1).reshape([x.size()[0], 1]) 
    D_x_1 = torch.prod(1.0 - x_temp, axis = 1).reshape([x.size()[0], 1]) 
    model_u_temp = D_x_0 * D_x_1 * net(x)
    return model_u_temp.reshape([x.size()[0], 1])

In [11]:
# loss function to DGM by auto differential
def loss_function(x):
#     x = generate_sample(data_size).cuda()
#     x.requires_grad = True
    u_hat = model(x)
    grad_u_hat = torch.autograd.grad(outputs = u_hat, inputs = x, grad_outputs = torch.ones(u_hat.shape), create_graph = True)
    laplace_u = torch.zeros([len(grad_u_hat[0]), 1])
    for index in range(dimension):
        p_temp = grad_u_hat[0][:, index].reshape([len(grad_u_hat[0]), 1])
        temp = torch.autograd.grad(outputs = p_temp, inputs = x, grad_outputs = torch.ones(p_temp.shape), create_graph = True, allow_unused = True)[0]
        laplace_u = temp[:, index].reshape([len(grad_u_hat[0]), 1]) + laplace_u
        part_2 = torch.sum((-laplace_u - f(x))**2)  / len(x)
    return part_2 

In [12]:
data_size = 10000
x = generate_sample(data_size)
x.requires_grad = True

In [13]:
# load model parameters
pretrained_dict = torch.load('net_params_DGM.pkl')
    
# get state_dict
net_state_dict = net.state_dict()

# remove keys that does not belong to net_state_dict
pretrained_dict_1 = {k: v for k, v in pretrained_dict.items() if k in net_state_dict}

# update dict
net_state_dict.update(pretrained_dict_1)

# set new dict back to net
net.load_state_dict(net_state_dict)

<All keys matched successfully>

In [14]:
param = {}
for name,parameters in net.named_parameters():
    print(name,':',parameters.size())
    param[name] = parameters.detach().numpy()

layer_out.weight : torch.Size([1, 1])
layer_out.bias : torch.Size([1])


In [15]:
param

{'layer_out.weight': array([[-2.00000119]]),
 'layer_out.bias': array([-0.99999869])}

In [16]:
for name, param in net.named_parameters():
    print(name, param)

layer_out.weight Parameter containing:
tensor([[-2.0000]], requires_grad=True)
layer_out.bias Parameter containing:
tensor([-1.0000], requires_grad=True)


In [17]:
dict = net.state_dict()
dict['layer_out.weight'] = torch.tensor([[-2]])
dict['layer_out.bias'] = torch.tensor([-1])
torch.save(dict, 'net_params_exact.pkl')

In [18]:
# load model parameters
pretrained_dict = torch.load('net_params_exact.pkl')
    
# get state_dict
net_state_dict = net.state_dict()

# remove keys that does not belong to net_state_dict
pretrained_dict_1 = {k: v for k, v in pretrained_dict.items() if k in net_state_dict}

# update dict
net_state_dict.update(pretrained_dict_1)

# set new dict back to net
net.load_state_dict(net_state_dict)

<All keys matched successfully>

In [19]:
for name, param in net.named_parameters():
    print(name, param)

layer_out.weight Parameter containing:
tensor([[-2.]], requires_grad=True)
layer_out.bias Parameter containing:
tensor([-1.], requires_grad=True)
